In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import numpy as np

In [3]:
seed = 1    # 난수고정

batch_size = 64
test_batch_size = 64

no_cuda = False

In [4]:
use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [5]:
torch.manual_seed(seed)

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)


test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1, 20 , 5, 1)
        self.conv2 = nn.Conv2d(20,50,5,1)
        self.fc1 = nn.Linear(4 * 4* 50,500)
        self.fc2 = nn.Linear(500,10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x,2,2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x,2,2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x,dim=1)

In [7]:
model = Net().to(device)

In [8]:
optimizer = optim.SGD(model.parameters(),lr=0.001,momentum=0.5)

In [9]:
# parameter shape 확인
params = list(model.parameters())
for i in range(8):
    print(params[i].size())

torch.Size([20, 1, 5, 5])
torch.Size([20])
torch.Size([50, 20, 5, 5])
torch.Size([50])
torch.Size([500, 800])
torch.Size([500])
torch.Size([10, 500])
torch.Size([10])


In [10]:
model.train

<bound method Module.train of Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)>

In [11]:
# model에 넣기 위한 첫 batch data 추출
data, target = next(iter(train_loader))

In [12]:
data.shape, target.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

In [13]:
data, target = data.to(device), target.to(device)

In [14]:
data.shape, target.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

In [15]:
# gradients 를 clear해서 새로운 최적화 값을 찾기 위해 준비
optimizer.zero_grad()

In [16]:
# 준비한 data를 model에 input으로 넣어 output을 얻음
output = model(data)

In [17]:
# Model에서 예측한 결과를 Loss Function에 넣음 Nagative Log-Likelihood Loss 
loss = F.nll_loss(output,target)

In [18]:
# Back Propagation을 통해 Gradients를 계산
loss.backward()

In [19]:
# 계산된 Gradients는 Parameter에 update
optimizer.step()

In [20]:
epochs =1
log_interval =100

In [21]:
for epoch in range(1, epochs+1):
    # Train mode
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % log_interval == 0 :
            print('Train Epoch: {} [ {}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100 * batch_idx / len(train_loader), loss.item()
            ))
    
    # 한 EPOCH에서 학습 후 바로 TEST 진행
    model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader : 
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim = True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test set: Average Loss: {:.4f}, Accuracy: {}/{} ( {:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), 100 * correct / len(test_loader.dataset)))

Train Epoch: 1 [ 0/60000 (0%)]	Loss: 2.320928
Train Epoch: 1 [ 6400/60000 (11%)]	Loss: 2.232707
Train Epoch: 1 [ 12800/60000 (21%)]	Loss: 2.129753
Train Epoch: 1 [ 19200/60000 (32%)]	Loss: 1.999541
Train Epoch: 1 [ 25600/60000 (43%)]	Loss: 1.665940
Train Epoch: 1 [ 32000/60000 (53%)]	Loss: 1.361088
Train Epoch: 1 [ 38400/60000 (64%)]	Loss: 0.853903
Train Epoch: 1 [ 44800/60000 (75%)]	Loss: 0.650938
Train Epoch: 1 [ 51200/60000 (85%)]	Loss: 0.642131
Train Epoch: 1 [ 57600/60000 (96%)]	Loss: 0.798663
Test set: Average Loss: 0.4794, Accuracy: 8655/10000 ( 87%)



In [22]:
# 앞에서 model.train() 모드로 변한 것처럼 평가 할 때는 model.eval()로 설정
model.eval()

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [23]:
# autograd engine, 즉 backpropagation이나 gradient 계산 등을 꺼서 memory usage를 줄이고 속도를 높임
test_loss = 0
correct = 0 

with torch.no_grad():
    data, target = next(iter(test_loader))
    data, target = data.to(device), target.to(device)
    output = model(data)

    test_loss += F.nll_loss(output, target, reduction='sum').item()
    # sum을 해줌으로 batch_size, shape에 관계없이 하나의 scalar로 return 하고 이를 더해준다.

    pred = output.argmax(dim = 1, keepdim = True)
    # keepdim을 이용해 차원수를 계속 유지

    correct = pred.eq(target.view_as(pred)).sum().item()

In [24]:
test_loss

33.3988151550293

In [25]:
output.shape

torch.Size([64, 10])

In [26]:
# pred와 target의 shape을 맞춰 비교, 동일하다는 것을 확인 가능
pred.shape, target.view_as(pred).shape

(torch.Size([64, 1]), torch.Size([64, 1]))

In [27]:
test_loss /= len(test_loader.dataset)

In [28]:
test_loss

0.0033398815155029297

In [29]:
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in test_loader : 
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += F.nll_loss(output, target, reduction='sum').item()
        pred = output.argmax(dim=1, keepdim = True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        
test_loss /= len(test_loader.dataset)

print('Test set: Average Loss: {:.4f}, Accuracy: {}/{} ( {:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset), 100 * correct / len(test_loader.dataset)))

Test set: Average Loss: 0.4794, Accuracy: 8655/10000 ( 87%)

